# Résolution de l'équation de Burgers à l'aide des méthodes GMRES

Dans ce travail, on se propose de simuler numériquement l'évolution de la solution d'une équation de Burgers, une équation aux dérivées partielles de dimension $1$. Plus précisément, nous voudrons simuler l'évolution de la solution de l'équation

$$ \left\{\begin{array}{ll}\partial_t u(t,x) + u(t,x)\partial_x u(t,x) = \mu \partial^2_x u(t,x), \forall (t,x)\in\mathbb{R}^+\times[0,1],\\ u(t,0) = u(t,1) = 0, \forall t\in\mathbb{R}^+, \\ u(0,x) = v(x), \forall x\in[0,1],\end{array}\right.$$

où $\mu>0$ est un coefficient de viscosité et $v(x) = \exp\left(-\frac{(x-x_0)^2}{d^2}\right)$ où $d = 0.1$ et $x_0 = 1/4$. Il est à noter que les solutions de ce type d'équation ont la fâcheuse tendance à développer des singularités en temps finis lorsque $\mu = 0$.

## 1. Implémentation des méthodes GMRES sur un système simple

On commence par l'implémentation de la méthode GMRES. Pour cela, on va débuter avec un problème de taille $n = 50$ afin de tester les méthodes et voir leur comportement. Pour cela, on va considérer le problème suivant:

$$ Ax=b,$$

où $A$ sera une matrice aléatoire que l'on construira à l'aide de la fonction définie dessous et $b\in\mathbb{R}^n$ sera tel que $b_i = 1$, $\forall i\in\{1,...,n\}$. 

On veillera à ce que les méthodes soient appliquées via des fonctions qui prendront en entrée une matrice $A\in M_{n,n}(\mathbb{R})$ (taille quelconque), un vecteur $b\in\mathbb{R}^n$, un point de départ $x_0\in\mathbb{R}^n$ et une tolérance pour le critère d'arrêt (que l'on fixera selon la méthode).

### 1.0. Chargement des librairies classiques pour le calcul scientifique et une fonction utile

In [ ]:
import numpy as np
import scipy as sp
import numpy.linalg as npl
import numpy.random as npr
import scipy.linalg as spl
import scipy.sparse as scs
import matplotlib.pyplot as plt
%matplotlib inline

# Generation d'une matrice aléatoire creuse de densité "d" et ayant ses valeurs propres (réelles) tirées 
# aléatoirement selon une loi normale de moyenne "m" et de variance "sigma"
def RandMat(n,d = 0.01,m = 1,sigma = 0.1):
    ev = np.sqrt(sigma)*npr.randn(n) + m
    D = np.diag(ev,0)
    A = scs.rand(n,n,d)
    A = scs.tril(A) + D
    return A

### 1.1. Rappel sur GMRES

Nous allons à présent implémenter les méthodes GMRES et CGNR. Tout d'abord, nous nous intéressons à la méthode GMRES. On rappelle que le principe de la méthode GMRES consiste à résoudre un problème de moindres carrés par une factorisation QR à l'aide de l'algorithme de Givens. Afin de rendre l'approche plus simple, nous allons décomposer la méthode.

Etant donné le résidu $r_0 = b - Ax_0$, la méthode GMRES consiste formellement à résoudre le problème de minimisation suivant, pour tout $k\geq 1$,

$$
y_k = \textrm{argmin}_{y\in\mathbb{R}^k}\left\| \|r_0\|_2e_{(k+1)}^1 - H_{(k+1,k)} y \right\|,
$$

où $e_{(k+1)}^1$ est le premier vecteur de la base canonique de $\mathbb{R}^{k+1}$ et $H_{(k+1,k)}$ est la matrice de type Hessenberg obtenue par l'algorithme d'Arnoldi. Cette dernière vérifie l'équation

$$
AQ_k = Q_{k+1} H_{(k+1,k)},
$$

avec $Q_k$ la matrice associée à une base orthonormal de l'espace de Krylov $\mathcal{K}_k(A,r_0)$. Une fois le vecteur $y_k$ calculé, une approximation de la solution du système linéaire est alors donnée par

$$
x_k = x_0 + Q_k y_k.
$$

Dans le notebook Krylov, vous avez implémenter la fonction `Step_Arnoldi` que nous donnons ci-dessous. On rappelle que cette fonction met en oeuvre l'ajout d'un vecteur $q^{k+1}$ à la base $Q_k$ et aussi l'ajout de la colonne $(h_{j,k})_{1\leq j\leq k+1}$ à la matrice $H_{(k,k-1)}$ pour $k\geq 2$. On remarque que l'on ne fera pas la première étape et, donc, que le vecteur $q^1$ (qui est ici donc $r_0/\|r_0\|_2$) sera déjà donné par l'utilisateur. Les arguments d'entrée sont:
- A: la matrice de la base de Krylov,
- Q: la matrice correspondant à la base $Q_k$,
- H: la matrice correspondant à la matrice $H_{(k,k-1)}$,

et les arguments de sortie:
- Q: la matrice correspondant à la base $Q_{k+1}$,
- H: la matrice correspondant à la matrice $H_{(k+1,k)}$.

Pour $k = 1$, la matrice $H_{(k,k-1)}$ n'est pas définie. On peut alors transmettre la variable d'entrée correspondante sous le format `None`.



In [ ]:
def Step_Arnoldi(A,Q,H):
    n,k = np.shape(Q)
    h = np.zeros((k+1,1))
    q = np.zeros((n,1))
    p = A.dot(Q[:,-1].reshape((n,1)))
    h[0:k,0] = Q.T.dot(p).squeeze()
    p -= Q.dot(h[0:k,0].reshape(k,1))
    h[k,0] = npl.norm(p)
    if H is None:
        H = h
    else:
        m = np.zeros((1,k-1))
        H = np.concatenate((H,m),axis = 0)
        H = np.concatenate((H,h),axis = 1)
    p = p/h[k,0]
    q[0:n,0] = p.squeeze()
    Q = np.concatenate((Q,q),axis = 1)
    return Q,H

### 1.2. Une première approche de GMRES

> **A faire :** À l'aide de la fonction `npl.lstsq`, implémentez la fonction `GMRES_v0` qui prend en arguments d'entrée la matrice `A` et le vecteur `b` du système linéaire que l'on souhaite résoudre ainsi que le vecteur initial `x0`, la tolérance `tol` et le nombre maximal d'itérations `itermax`. Les arguments de sortie se borneront à la solution approximée `x` du système linéaire. ** Attention :** Prenez bien garde à la dimension des vecteurs qui seront sous la forme `(n,1)` où `n` est la longueur du vecteur (et non pas `(n,)`). On utilisera la fonction `reshape` pour parvenir à ce format.

In [ ]:
def GMRES_v0(A,b,x0 = None,tol = 1e-10,itermax = None):
    if x0 is None:
        x0 = np.zeros((len(b),1))
    if itermax is None:
        itermax = len(b)
    b = b.reshape((len(b),1))
    r = b - A.dot(x0)
    beta = npl.norm(r)
    rho = beta
    Q = r.reshape((len(b),1))/float(beta)
    H = None
    niter = 1
    while (niter<itermax) and (rho>tol):
        Q,H = Step_Arnoldi(A,Q,H)
        e1 = np.zeros(niter+1)
        e1[0] = 1
        y = npl.lstsq(H,e1*beta)[0]
        y = y.reshape(niter,1)
        x = x0 + Q[:,:niter].dot(y)
        r = b - A.dot(x)
        rho = npl.norm(r)
        niter += 1
    return x

> **A faire :** Testez votre code avec un système linéaire de taille 50.

In [ ]:
A = RandMat(50)
b = np.random.random(50)
x_exact = npl.solve(A,b)
x = GMRES_v0(A,b)
print(npl.norm(x-x_exact.reshape(50,1)))

### 1.3. Résolution du problème de moindres carrés avec Givens

Nous allons maintenant nous passer de la boîte noire `npl.lstsq` en solutionnant le problème de moindres carrés par une factorisation QR de la matrice de Hessenberg `H`. Nous connaissons différentes approches afin d'obtenir une factorisation QR et nous allons ici nous baser sur la méthode de Givens puisque l'on veut simplement éliminer la sous-diagonale de la matrice `H`. On rappelle que, pour résoudre un problème de moindres carrés à l'aide d'une factorisation QR, on va factoriser la matrice `H`

$$
H = O R,
$$

où la matrice $O$ est orthogonal et la matrice $R$ est triangulaire supérieure. Dans le cadre de la méthode de Givens, l'application des rotations va se traduire par l'application de la matrice $O^T$ à la matrice $H$. Ainsi, pour résoudre un problème de moindres carrés de la forme

$$
\min_{y\in\mathbb{R}^k}\|e-Hy\|_2,
$$

on va se borner à appliquer les rotations à la matrice $H$ et au vecteur $e$ pour obtenir le système

$$
O^T H y = O^Te \quad\Leftrightarrow\quad R y = f.
$$

où $f = O^Te$. La matrice $R$ est ici de la même taille que $H$ mais sa dernière ligne est nulle. 

> **A faire :** Implémentez la fonction `QR_Givens` qui prendra en entrée la matrice `H` et le vecteur `e` et qui renvoie la matrice `R` et le vecteur `f` selon la méthode de Givens (c'est-à-dire à l'aide de rotations élémentaires dans le plan).

In [ ]:
def QR_Givens(H,e):
    n,m = np.shape(H)
    e = e.reshape(len(e),1)
    A = np.concatenate((H,e),axis = 1)
    for k in range(m):
        nm = np.sqrt(A[k,k]**2 + A[k+1,k]**2)
        c = A[k,k]/nm
        s = A[k+1,k]/nm
        v_tmp = np.copy(A[k,:])
        A[k,:] = c*A[k,:] + s*A[k+1,:]
        A[k+1,:] = c*A[k+1,:] - s*v_tmp
    R = A[:m+1,:m]
    f = A[:,m]
    return R,f

> **A faire :** Testez votre code avec un problème de moindres carrés où la matrice est de type Hessenberg.

In [ ]:
A = np.random.random((6,5))
A = np.triu(A,-1)
b = np.random.random(6)

R,f = QR_Givens(A,b)
x = npl.solve(R[:5,:5],f[:5])
x_exact = npl.lstsq(A,b)[0]
print(npl.norm(x_exact.squeeze() - x))

> **A faire :** Implémentez la fonction `GMRES_v1` qui reprend la fonction `GMRES_v0` mais où vous utilisez la fonction `QR_Givens` et non plus la fonction `npl.lstsq`. On pourra utiliser la fonction `npl.solve` pour résoudre le système triangulaire.

In [ ]:
def GMRES_v1(A,b,x0 = None,tol = 1e-10,itermax = None):
    if x0 is None:
        x0 = np.zeros((len(b),1))
    if itermax is None:
        itermax = len(b)
    b = b.reshape((len(b),1))
    r = b - A.dot(x0)
    beta = npl.norm(r)
    rho = beta
    Q = r.reshape((len(b),1))/float(beta)
    H = None
    niter = 1
    while (niter<itermax) and (rho>tol):
        Q,H = Step_Arnoldi(A,Q,H)
        e1 = np.zeros(niter+1)
        e1[0] = 1
        R,f = QR_Givens(H,e1*beta)
        rho = abs(f[-1])
        niter += 1
    y = npl.solve(R[:niter-1,:niter-1],f[:niter-1]).reshape(niter-1,1)
    x = x0 + Q[:,:niter-1].dot(y)
    return x

> **A faire :** Testez votre code avec un système linéaire de taille 50.

In [ ]:
A = RandMat(50)
b = np.random.random(50)
x_exact = npl.solve(A,b)
x = GMRES_v1(A,b)
print(npl.norm(x-x_exact.reshape(50,1)))

### 1.4. La version optimisée de GMRES

Nous allons à présent optimiser les opérations de `GMRES_v1`. Une première étape va consister à ne plus stocker la matrice `H` obtenue par la fonction `Step_Arnoldi` mais uniquement `T`, la matrice triangulaire supérieure obtenue après la factorisation QR par la méthode de Givens. Pour cela, on va implémenter 2 nouvelles fonctions: 

- `Step_Arnoldi_h` qui va, à partir des matrices `A` et `Q`, donner la nouvelle matrice `Q` (avec un vecteur de la base de Krylov supplémentaire) et le vecteur `h` (et non plus la matrice `H`) qui correspond à la nouvelle colonne que l'on aurait obtenue dans la matrice `H` avec le procédé d'Arnoldi classique,
- `QR_Givens_h` qui va prendre en argument une liste `O` qui contiendra les coefficients de rotations dans le plan, `h` un vecteur et `g` un autre vecteur. Cette fonction va, dans un premier temps, appliquer l'ensemble des rotations contenues dans `O` au vecteur `h`. Puis, dans un second temps, elle va calculer la rotation permettant d'éliminer le dernier coefficient du nouveau vecteur `h`, appliquer cette rotation au vecteur `h` et au vecteur `g` puis ajouter les coefficients de cette rotation à la liste `O`. On reprendra garde à gérer le cas où `O` est vide.

> **A faire :** Implémenter les fonctions `Step_Arnoldi_h` et `QR_Givens_h`.

In [ ]:
def Step_Arnoldi_h(A,Q):
    n,k = np.shape(Q)
    h = np.zeros((k+1,1))
    q = np.zeros((n,1))
    p = A.dot(Q[:,-1].reshape((n,1)))
    h[0:k,0] = Q.T.dot(p).squeeze()
    p -= Q.dot(h[0:k,0].reshape(k,1))
    h[k,0] = npl.norm(p)
    p = p/h[k,0]
    q[0:n,0] = p.squeeze()
    Q = np.concatenate((Q,q),axis = 1)
    return Q,h

def QR_Givens_h(O,h,g):
    if O is not None:
        k = len(O)
        for i in range(k):
            cf_tmp = float(h[i])
            h[i] = O[i][0]*h[i] + O[i][1]*h[i+1]
            h[i+1] = O[i][0]*h[i+1] - O[i][1]*cf_tmp
    nm = npl.norm(h[-2:])
    c = float(h[-2])/nm
    s = float(h[-1])/nm
    cf_tmp = float(h[-2])
    h[-2] = c*h[-2] + s*h[-1]
    h[-1] = c*h[-1] - s*cf_tmp    
    cf_tmp = float(g[-2])
    g[-2] = c*g[-2] + s*g[-1]
    g[-1] = c*g[-1] - s*cf_tmp  
    if O is None:
        O = [[c,s]]
    else:
        O += [[c,s]]
    return O,h,g

> **A faire :** Tester les fonctions avec les données suivantes et vérifier que vous obtenez les bonnes valeurs.

In [ ]:
A = np.array([[1.,5.],[-3.,2.]])
b = np.array([1.,-1.]).reshape((2,1))
g = np.array([npl.norm(b),0.])
Q = b/npl.norm(b)
O = None
Q,h_tmp = Step_Arnoldi_h(A,Q)
h = np.copy(h_tmp)
O,h,g = QR_Givens_h(O,h,g)
print("Vecteur h:",h)
print("Coefficients de rotations:",O[0][0],"et",O[0][1])

On peut finalement passer à l'implémentation de la fonction `GMRES_v2` qui va exploiter les fonctions `Step_Arnoldi_h` et `QR_Givens_h` afin de ne plus stocker la matrice `H` et de ne plus refaire, à chaque itération, la factorisation QR complète de `H`. On reprendra soin de bien définir un vecteur `g` qui sera initialisé comme un scalaire dont la valeur est la norme de $r_0$ et auquel on ajoutera, à chaque itération, un coefficient $0$ (voir `np.vstack`) puis on le passera en argument à la fonction `QR_Givens_h` (comme 3ème argument).

> **A faire :** Implémenter la fonction `GMRES_v2`.

In [ ]:
def GMRES_v2(A,b,x0 = None,tol = 1e-10,itermax = None):
    if x0 is None:
        x0 = np.zeros((len(b),1))
    if itermax is None:
        itermax = len(b)
    b = b.reshape((len(b),1))
    r = b - A.dot(x0)
    beta = npl.norm(r)
    rho = beta
    g = np.array([beta])
    Q = r.reshape((len(b),1))/float(beta)
    O = None
    niter = 1
    while (niter<itermax) and (rho>tol):
        g = np.vstack((g, 0.))
        Q,h = Step_Arnoldi_h(A,Q)
        O,h,g = QR_Givens_h(O,h,g)
        if niter == 1:
            T = h[-2].reshape((1,1))
        else:
            m = np.zeros((1,niter-1))
            T = np.concatenate((T,m),axis = 0)
            T = np.concatenate((T,h[:-1]),axis = 1)
        rho = abs(g[-1])
        niter += 1
    y = npl.solve(T,g[:-1]).reshape(niter-1,1)
    x = x0 + Q[:,:niter-1].dot(y)
    return x

> **A faire :** Testez votre code avec un système linéaire de taille 50.

In [ ]:
A = RandMat(50)
b = np.random.random(50)
x_exact = npl.solve(A,b)
x = GMRES_v2(A,b)
print(npl.norm(x-x_exact.reshape(50,1)))

## 2. L'équation de Burgers

On s'intéresse maintenant à l'équation de Burgers qui est donnée par

$$ \left\{\begin{array}{ll}\partial_t u(t,x) + u(t,x)\partial_x u(t,x) = \mu \partial^2_x u(t,x), \forall (t,x)\in ]0,T]\times[0,1],\\ u(t,0) = u(t,1) = 0, \forall t\in\mathbb{R}^+, \\ u(0,x) = v(x), \forall x\in[0,1].\end{array}\right.$$

On commence par discrétiser l'équation en temps. Pour cela, on choisit un schéma d'Euler semi-implicite qui apporte un compromis entre Euler explicite et Euler implicite. Pour un pas en temps uniforme $s>0$, celui-ci nous amène à l'équation discrétisée en temps suivante

$$\left\{\begin{array}{ll} u_{n+1}(x) - u_n(x) + s u_n(x)\partial_x u_{n+1}(x) = s\mu \partial^2_x u_{n+1}(x), \quad\forall x\in[0,1],\\ u_{n+1}(0) = u_{n+1}(1) = 0,\\ u_0(x) = v(x), \forall x\in[0,1].\end{array}\right.$$

On passe maintenant à la discrétisation en espace. L'intervalle $[0,1]$ est discrétisé en $n+2$ points ($2$ points consistuent les bords). La discrétisation de l'opérateur de Laplace se fait par des différences finies centrées du second ordre et de pas $h = \frac 1 {n+1}$ (avec conditions aux bords de Dirichlet). On en déduit la matrice $L\in M_{n,n}(\mathbb{R})$ associée

$$ (L)_{i,j} =\left\{\begin{array}{ll} -\frac 2 {h^2} \mbox{ lorsque } i=j,\\ \frac1{h^2} \mbox{ lorsque } i=j+1 \mbox{ ou } j = i+1\\ 0 \mbox{ sinon.} \end{array}\right. $$

Pour l'opérateur gradient, on peut choisir des différences finies du second ordre centrées, à droite ou à gauche. On obtient $G_0$, $G_1$ et $G_{-1}\in M_{n,n}(\mathbb{R})$ données par:

$$ (G_0)_{i,j} =\left\{\begin{array}{ll} -\frac1{2h} \mbox{ lorsque } i=j+1,\\ \frac1{2h} \mbox{ lorsque } j=i+1,\\ 0 \mbox{ sinon.} \end{array}\right. $$

$$ (G_1)_{i,j} =\left\{\begin{array}{ll} -\frac3{2h} \mbox{ lorsque } j=i,\\ \frac2{h} \mbox{ lorsque } j=i+1,\\ -\frac1{2h} \mbox{ lorsque } j=i+2,\\ 0 \mbox{ sinon.} \end{array}\right. $$
et
$$ (G_{-1})_{i,j} =\left\{\begin{array}{ll} \frac3{2h} \mbox{ lorsque } i=j,\\ -\frac2{h} \mbox{ lorsque } i=j+1,\\ \frac1{2h} \mbox{ lorsque } i=j+2,\\ 0 \mbox{ sinon.} \end{array}\right. $$

L'équation de Burgers discrétisée en temps et en espace devient donc
$$\left\{\begin{array}{ll} u_{n+1} - u_n + s U_n G_k u_{n+1} = s\mu L u_{n+1},\\ u_0 = v,\end{array}\right.$$

où $k\in \{-1,0,1\}$ et où l'on note $U_n$ la matrice diagonale de taille $n$ telle que
$$ \left(U_n\right)_{i,j} = \left\{\begin{array}{ll} (u_n)_i \mbox{ lorsque } i=j,
\\ 0 \mbox{ sinon.}\end{array}\right. $$ 

On peut réécrire la première équation sous la forme d'un système linéaire
$$ \left(\mbox{Id} + sU_nG_k - s\mu L\right) u_{n+1} = u_n ,$$ 

dont la solution est l'itérée suivante dans notre schéma d'Euler semi-implicite.

### 2.0. Définition des matrices et de la donnée initiale

In [ ]:
def v(x):
    d = 1e-1
    return np.exp(-(x-0.25)**2/d**2)
def Laplace_matrix(n):
    return -scs.diags(2*np.ones(n),0) + scs.diags(np.ones(n-1),1) + scs.diags(np.ones(n-1),-1)
def Gradient_matrix(n,k=0):
    if (k == 0):
        G = scs.diags(-0.5*np.ones(n-1),-1) + scs.diags(0.5*np.ones(n-1),1)
    elif (k==1):
        G = scs.diags(-1.5*scs.ones(n),0) + scs.diags(2*np.ones(n-1),1) + scs.diags(-0.5*np.ones(n-2),2)
    elif (k==-1):
        G = scs.diags(1.5*np.ones(n),0) + scs.diags(-2*np.ones(n-1),-1) + scs.diags(0.5*np.ones(n-2),-2)
    return G
def Un_matrix(un):
    return scs.diags(un[:,0],0)

### 2.1. Simulation de l'équation de Burgers

Au vu des performances de la méthode CGNR (et de sa sensibilité au conditionnement), on se propose de n'utiliser que la méthode GMRES pour résoudre le système linéaire. D'autre part, on pourra se baser sur les paramètres suivants:
- $h = 1/(n+1)$ avec $n = 200$,
- $T = 2$, $s = T/m$ avec $m = 200$,
- $\mu = 10^{-3}$,
- $k = 0$ (différences finies centrées pour le gradient).

Afin de tracer la solution, on pourra utiliser la fonction $plt.pcolor$ afin de faire un tracer de la solution en $2$ dimensions: le temps et l'espace. Enfin, il est intéressant de tester l'influence de la valeur du paramètre $\mu$ lorsque celui-ci devient petit en fonction des $3$ discrétisations de l'opérateur gradient.

In [ ]:
def Burgers(v,n,m,T = 1,mu = 1e-7, k = 0):
    x = np.linspace(0,1,n+2)
    u = v(x)
    x = x[1:n+1]
    u = u[1:n+1].reshape((n,1))
    U = np.copy(u)
    h = 1./(n+1)
    s = float(T)/m
    L = Laplace_matrix(n)
    G = (s/h)*Gradient_matrix(n,k)
    I = np.eye(n)
    A0 = I - (mu*s/h**2)*L
    t = np.linspace(0,T,m+1)
    for i in range(m):
        A = A0 + Un_matrix(np.array(u))*G
        u = GMRES_v2(A,u)
        U = np.concatenate((U,np.copy(u)),axis = 1)
    T,X = np.meshgrid(t,x)
    return U,T,X

In [ ]:
n = 200
m = 200

U,T,X = Burgers(v,n,m,T = 5,k=-1)
plt.pcolor(T,X,np.array(U))
plt.show()